<a href="https://colab.research.google.com/github/dhivyapm/divya/blob/master/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I found the information about how the model works in this website
https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01

In [1]:
from google.colab import files
upload = files.upload()

Saving satelite.csv to satelite.csv


In [2]:
import pandas as pd # to read the dataset and convert the dataset to DataFrame which is in table format
import numpy as np # numpy is for computing, here I'm using this package for randomly select the rows from the dataset
import random # to randomly select the datapoints from dataset
df = pd.read_csv('/content/satelite.csv')
print(df.shape)

(6435, 37)


In [3]:
features = ['Column17','Column18','Column19','Column20']
random.seed(30) # If you want to generate the same number every time,before doing the shuffle
df = df.sample(frac=1)# shuffle the dataset
df.shape

(6435, 37)

In [4]:
#train_test_Split
train_size = int(0.9 * len(df)) #train_size =5791
X_train = df[features][:train_size] # X_train takes the values of the columns from 1 to 36 from the dataset upto the train_size rows(5791 rows)
y_train = df['Column37'][:train_size] #y_train takes the last column values which is label upto the train_size(5791 rows)
X_test = df[features][train_size:] # X_test takes the values of the columns from 1 to 36 starting from rows 5791 upto the dataset length which is 6435
y_test = df['Column37'][train_size:] # y_test takes the last column values which is label starting from rows 5791 upto the dataset length which is 6435
print(X_train)

      Column17  Column18  Column19  Column20
2547        86       104       108        89
4316        74        88        97        83
1602        57       106       110        90
4063        92       120       120        98
456         57        63        63        51
...        ...       ...       ...       ...
5392        56        79        96        78
334         92       112       118        92
816         64        73        74        57
5409        66       109       122        96
5919        64        81        90        83

[5791 rows x 4 columns]


In [0]:
#calculating the bootstrap that is taking sample from train set and left out samples in dataset will be taken as out-of-bag which is used to test the model performance
def n_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True)) #it as the index of the sample selected from the training set
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices] # it as the index of the left over samples in the training set
    X_bootstrap = X_train.iloc[bootstrap_indices].values # here it takes the values of the specific index from the training
    y_bootstrap = y_train.iloc[bootstrap_indices].values # it takes the array elements of the specific indices(labels)
    X_oob = X_train.iloc[oob_indices].values # it takes the values of the left over samples in the training set
    y_oob = y_train.iloc[oob_indices].values # it takes the array elements from the left over samples in the training set(labels)
    return X_bootstrap, y_bootstrap, X_oob, y_oob # returns all the values of bootstrap and oob

In [6]:
X_bootstrap,y_bootstrap,X_oob,y_oob = n_bootstrap(X_train,y_train)
print('X_bootstrap:\n',X_bootstrap,'\ny_bootstrap:\n',y_bootstrap)
print('X_oob:\n',X_oob,'\ny_oob:',y_oob)

X_bootstrap:
 [[ 60  62  67  57]
 [ 53  91 104  88]
 [ 41  29 113 129]
 ...
 [ 64  77  82  65]
 [ 52  46  80  63]
 [ 67  79  85  65]] 
y_bootstrap:
 [7 1 2 ... 7 5 7]
X_oob:
 [[ 86 104 108  89]
 [ 74  88  97  83]
 [ 53  87 104  85]
 ...
 [ 93 107 113  92]
 [ 63 102 114  90]
 [ 64  73  74  57]] 
y_oob: [3 7 1 ... 3 1 7]


In [0]:
# calculate the Entropy(measure of disorder)
#To which group does this sample belongs to
#Entropy controls how a Decision Tree decides to split the data.
#where entropy takes in a probability of a class within a node 
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return  - (p * np.log2(p) + (1 - p) * np.log2(1-p))
         # these formula provided in these https://medium.com/coinmonks/what-is-entropy-and-why-information-gain-is-matter-4e85d46d2f01
         # in that they clearly explained the concept of entropy and Information-Gain

In [0]:
#calculate the information gain to compare the entropy before and after split
#Information_gain takes in a list of the classes from the left and right child and returns the information gain of that particular split.
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent) #calculate the entropy of parent node
    IG_l = entropy(p_left) #calculate the entropy of left child node
    IG_r = entropy(p_right) #calculate the entropy of right child node
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r #return the information gain of parent and childs

In [0]:
#function for finding the split point for the decision-tree based on the highest information gain
#select random features from the bootstrap 
#for each feature selected, iterate through each value in the X_bootstrap and compute the information gain
#Return a array of values that gives the highest information gain, which will represent a node in a tree consist of 
#index of the feature,split-value,left-child and right-child node.
def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])
    
    while len(feature_ls) <= max_features:
        feature_idx = random.sample(range(num_features), 1)
        if feature_idx not in feature_ls:
            feature_ls.extend(feature_idx)
    
    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
        for split_point in X_bootstrap[:,feature_idx]:
            left_child = {'X_bootstrap': [], 'y_bootstrap': []}
            right_child = {'X_bootstrap': [], 'y_bootstrap': []}
            
            # split children for continuous variables
            if type(split_point) in [int, float]:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value <= split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
                  # split children for categoric variables
            else:
                for i, value in enumerate(X_bootstrap[:,feature_idx]):
                    if value == split_point:
                        left_child['X_bootstrap'].append(X_bootstrap[i])
                        left_child['y_bootstrap'].append(y_bootstrap[i])
                    else:
                        right_child['X_bootstrap'].append(X_bootstrap[i])
                        right_child['y_bootstrap'].append(y_bootstrap[i])
                
            split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
            if split_info_gain > best_info_gain:
                best_info_gain = split_info_gain
                left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
                right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
                node = {'information_gain': split_info_gain, 
                        'left_child': left_child, 
                        'right_child': right_child, 
                        'split_point': split_point,
                        'feature_idx': feature_idx}
                
    
    return node            

In [0]:
#function to tell the node when to stop splitting
def split_node(node, max_features, min_samples_split, max_depth, depth):
  left_child = node['left_child']       #for given node left-child node to left-child
  right_child = node['right_child']     #for given node right-child node to right-child

  del(node['left_child'])              #delete the left-node after copying to left-child
  del(node['right_child'])             #delete the right-node after copying to right-child

  if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0: # check if either of the children has observed '0' that means it is pure
    empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
    node['left_split'] = terminal_node(empty_child) #no split further create a left-child and return the tree
    node['right_split'] = terminal_node(empty_child) #no split further create a right-child and return the tree
    return

  if depth >= max_depth: # Check current depth of the tree has reached the maximum depth if true
    node['left_split'] = terminal_node(left_child) #create a left-child and return the tree
    node['right_split'] = terminal_node(right_child) # create a right-child and return tree
    return node
    #Check if number of observations in the left child  is less than the minimum samples needed to make a split
    #if so return the tree else call the function find_split_point() and repeat the split-node until each branch as terminal node
  if len(left_child['X_bootstrap']) <= min_samples_split: #here checks for the left-child with min-samples
    node['left_split'] = node['right_split'] = terminal_node(left_child)
  else: # if it is not less than min-split repeat the process of splitting until it reaches the min-split
    node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
    split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
  if len(right_child['X_bootstrap']) <= min_samples_split: #here checks the right-child with min-samples
    node['right_split'] = node['left_split'] = terminal_node(right_child)
  else:
    node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
    split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1) #finally repeats the function until each branch as the terminal node

In [0]:
#function for deciding the terminal-node (leaf-node)
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    return max(y_bootstrap, key = y_bootstrap.count)

In [0]:
#function to build the tree with the boostrap values
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features) #creates the node with parent,right and left-child node
# Once we have a root node we will use the split_node() which will recusively split each internal node until each branch reached terminal node
    split_node(root_node, max_features, min_samples_split, max_depth, 1) 
    return root_node

In [0]:
root_node = build_tree(X_bootstrap,y_bootstrap,10,2,2)

In [15]:
root_node

{'feature_idx': 3,
 'information_gain': 0.026250135035698285,
 'left_split': {'feature_idx': 0,
  'information_gain': 0.12282310333601809,
  'left_split': {'feature_idx': 3,
   'information_gain': 0.0,
   'left_split': 1,
   'right_split': 1,
   'split_point': 96},
  'right_split': {'feature_idx': 0,
   'information_gain': 0.14393663941894064,
   'left_split': {'feature_idx': 2,
    'information_gain': 0.0,
    'left_split': {'feature_idx': 3,
     'information_gain': 0.0,
     'left_split': 1,
     'right_split': 1,
     'split_point': 96},
    'right_split': {'feature_idx': 3,
     'information_gain': 0.0,
     'left_split': 1,
     'right_split': 1,
     'split_point': 96},
    'split_point': 123},
   'right_split': {'feature_idx': 1,
    'information_gain': 0.08569171658202301,
    'left_split': {'feature_idx': 2,
     'information_gain': 0.0,
     'left_split': 3,
     'right_split': {'feature_idx': 0,
      'information_gain': 0.0,
      'left_split': {'feature_idx': 0,
       'i